# <font color='black'><center><center></font> 
![title](img/All.png) 
# <font color='white'><center><center></font> 
# <font><center>Artificial Intelligence<center></font>
## <font ><center>Hospital Job Scheduling<center></font>
## <center>Teacher: Dr. Fadaei<center>
### <center>Student Name: Mohammadali Shakerdargah<center>
### <center>Student Number: 810098002<center>


### <font color='White'>Purpose:</font>
    In this project we are going to use "Genetic Algorithm" to make a schedual for a hospital
    
### <font color='white'>What  we did in this Hands-On:</font>
    In this Hands-On we used "Genetic Algorithm" made an optimum schedual for a hospital which is currently conflicted and faced Corona-Virus.
    
#### <font>Stage1:</font>
        At first we generated 300 possible abswers which were complete but not consistent, we faced them as choromosomes. Each choromosom represents a solution and the population of these choromosoms will be our generation.
#### <font>Stage2:</font>
        We Write a function that calculates Fitness, this presents how good this chromosom(solution) is, after comparing all choromosoms and ranking them, we can have the best choromosom of this generation, if this choromosom did not satisfy our expectations, we will need to make a new generation and do all of this again and again till it finally satisfies our expectation which is clearified in the project description
        

### <font>Test Files:</font>

In [3]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

### <font>Reading Test File Content:</font>

In [4]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

### <font>Job Schedualer:</font>

In [5]:
import random
!jt -t onedork -T -N -kl

'jt' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        # You can use these values for your genetic algorithm
        self.popSize = 300
        self.elitismPercentage = 16 #(move 16% best of parents directly to the new population)
        self.pc = 0.8 #(crossover probability)
        self.pm = 0.2  #(mutation probability)
        
        
        self.chromosomes = self.generateInitialPopulation()
        
    def generateInitialPopulation(self): # list( [ [Day_1],[Noon_1],[Night_1],....[Night_n]] )
        population = []
        for i in range(self.popSize):
            newChromosome = [[0 for x in range(0)] for y in range((self.days)*3)] 
            for j in range((self.days)*3):
                numDoctorsOnShift = random.randrange(0,self.doctors)
                for k in range(numDoctorsOnShift):
                    docNum = random.randrange(0,(self.doctors))
                    if(docNum not in newChromosome[j]):
                        newChromosome[j].append(docNum)
            population.append(newChromosome)
        return population
    
    def crossOver(self, chromosome1, chromosome2): # Coin Method - Happens with 80% probability
        child1 = []
        child2 = []
        for i in range(len(chromosome1)):
            coin = random.randrange(0,2) # 0 or 1
            if(coin == 1):
                child1.append(chromosome1[i])
                child2.append(chromosome2[i])
            else:
                child2.append(chromosome1[i])
                child1.append(chromosome2[i])
        return child1, child2
                
    def mutate(self, chromosome): # Swap Method - Happens with 20% probability
        shiftOfDay1 = random.randrange(0,len(chromosome))
        shiftOfDay2 = random.randrange(0,len(chromosome))
        chromosome[shiftOfDay1], chromosome[shiftOfDay2] = chromosome[shiftOfDay2], chromosome[shiftOfDay1] 
        return chromosome
        
        
    def calculateFitness(self, chromosome):
        score=0
        shift=0
        day=0
        
        for i in range(len(chromosome)):
            
            if((self.allShifts[day][shift][0]) <= len(chromosome[i])
                  and len(chromosome[i]) <= (self.allShifts[day][shift][1])): # min-max each shift
                score += 1
            shift = shift+1
            if(((i+1)%3 == 0) and (i != 0)):
                shift = 0
                day = day+1
            
            
            
            if(((i+1)%3 == 0) and (i != 0) and (i != len(chromosome)-1)): # Night Shifts -Not firstDay&lastNight
                countDocs=0
                for j in range(len(chromosome[i])):#Not day-Noon shift after one night 
                    if( (chromosome[i][j] not in chromosome[i+1] ) 
                       and (chromosome[i][j] not in chromosome[i+2])): 
                           countDocs += 1
                if(countDocs == len(chromosome[i])):
                    score += 1
                    
                countDocs=0
                if(i <= len(chromosome)-6): #No 3 Nights in a row 
                    for j in range(len(chromosome[i])):
                        if( (chromosome[i][j] not in chromosome[i+3] ) 
                           or (chromosome[i][j] not in chromosome[i+6])):
                               countDocs += 1
                    if(countDocs == len(chromosome[i])):
                        score += 1              
                                
        for doc in range(self.doctors): #Limit shifts for any doctor
            shiftsTakenForDoc = 0
            for j in range(len(chromosome)):
                shiftsTakenForDoc += chromosome[j].count(doc)
            if(shiftsTakenForDoc <= self.maxCapacity):
                score += 1
                
        return score
    
    def rankPopulation(self):
        rankedPopulation = []
        for i in range(len(self.chromosomes)):
            rankedPopulation.append([self.calculateFitness(self.chromosomes[i]), self.chromosomes[i]])
        rankedPopulation = sorted(rankedPopulation,key=lambda x: (x[0]), reverse = True)
        return rankedPopulation
    
    def generateNewPopulation(self):
        rankedPopulation = self.rankPopulation()
        straighToNextGen = int(self.elitismPercentage*((self.popSize)/100)) #48 best
        nextGen = []
        for i in range(straighToNextGen):
            nextGen.append(rankedPopulation[i][1])
        for i in range(straighToNextGen,(self.popSize)-1):
            crossOverChance = random.uniform(0,1)
            mutateChance = random.uniform(0,1)
            if((0 <= crossOverChance) and (crossOverChance <= self.pc)): #Can crossOver 
                child1, child2 = self.crossOver(rankedPopulation[i][1],rankedPopulation[i+1][1])
                if((0 <= crossOverChance) and (crossOverChance <= self.pm)): #Can mutate 
                    child1 = self.mutate(child1)
                    child2 = self.mutate(child2)
                nextGen.append(child1)
                nextGen.append(child2)
            elif((0 <= crossOverChance) and (crossOverChance <= self.pm)): #Can only mutate 
                child1 = self.mutate(rankedPopulation[i][1])
                child2 = self.mutate(rankedPopulation[i+1][1])
                nextGen.append(child1)
                nextGen.append(child2)
            else: #Nothing happens
                nextGen.append(rankedPopulation[i][1])
                nextGen.append(rankedPopulation[i+1][1])
                
        return nextGen
    
    def schedule(self):
        goalScore = ((self.days)*3) + ((self.days)-1) + ((self.days)-2) + (self.doctors)  #-1CozNeverGetsToLastNight                            
        print("goalScore: ",goalScore)
        stateRate = self.rankPopulation()[0][0]
        while (stateRate < goalScore):
            self.chromosomes = self.generateNewPopulation()
            stateRate = self.rankPopulation()[0][0]
#             print(stateRate)
        print("Reached Score: ",self.rankPopulation()[0][0])
        print("Schedule: ",self.rankPopulation()[0][1])
        return self.rankPopulation()[0][1]

### <font>Execution: First File</font>

In [54]:
import time

fileInfo1 = readInput(testFile1)
timeAll = []
for i in range(2):
    start = time.time()
    scheduler = JobScheduler(fileInfo1)
    sol1 = scheduler.schedule()
    end = time.time()
    timeAll.append((end - start))
    print("\n")
print("test 1: ", '%.2f'%(sum(timeAll)/2), 'sec')

goalScore:  54
Reached Score:  54
Schedule:  [[6, 2], [0, 5], [2, 0, 3], [1], [5, 4], [5, 2], [6], [3], [4], [0], [1], [0, 1, 3], [6, 4], [6], [1], [6], [6], [4], [3, 5], [6], [1], [], [4, 0], [2, 1], [], [], [], [3], [3], [2, 6, 3]]


goalScore:  54
Reached Score:  54
Schedule:  [[5, 2], [2, 0], [6, 3, 2], [4], [0, 4], [0], [1], [4], [4], [6], [2], [6, 2, 4], [3], [], [3], [1], [], [4, 2], [6, 1], [0], [5], [4], [0], [2, 3], [0], [6], [], [0], [1], [3, 5]]


test 1:  3.00 sec


In [55]:
for i in range(len(sol1)):
    print(sol1[i])
    if((i+1)%3 ==0):
        print('\n')

[5, 2]
[2, 0]
[6, 3, 2]


[4]
[0, 4]
[0]


[1]
[4]
[4]


[6]
[2]
[6, 2, 4]


[3]
[]
[3]


[1]
[]
[4, 2]


[6, 1]
[0]
[5]


[4]
[0]
[2, 3]


[0]
[6]
[]


[0]
[1]
[3, 5]




In [69]:

f1 = open('output1.txt','w')
for i in range(len(sol1)):
    if(len(sol1[i]) == 0):
        f1.write("empty ")
        if((i+1)%3 ==0):
            f1.write("\n")
        continue
    for j in range(len(sol1[i])):
        if(j != len(sol1[i])-1):
            f1.write(str(sol1[i][j])+',')
        else:
            f1.write(str(sol1[i][j]))
    f1.write(" ")
    if((i+1)%3 ==0):
        f1.write("\n")

f1.close()

### <font>Execution: Second File</font>

In [52]:
fileInfo2 = readInput(testFile2)

timeAll = []
for i in range(2):
    start = time.time()
    scheduler = JobScheduler(fileInfo2)
    sol2 = scheduler.schedule()
    end = time.time()
    timeAll.append((end - start))
    print("\n")
print("test 2: ", '%.2f'%(sum(timeAll)/2), 'sec')

goalScore:  81
Reached Score:  81
Schedule:  [[8, 0], [6], [2, 0], [6, 8, 5, 7], [5], [3], [0], [7], [8, 1], [7, 6, 0], [4, 2, 5], [1, 0], [4], [], [4], [7, 8], [8, 3], [3, 4], [], [2, 8], [8, 5], [2, 7, 6], [2, 3, 0], [0, 6], [4], [4, 2, 8, 1, 7], [0, 3], [8], [5, 7], [5], [4, 6], [2, 0], [], [3, 5], [3], [], [7, 4, 3, 2], [4, 2], [5, 2], [7], [1, 0], [4, 0, 7], [3], [2, 8], [5]]


goalScore:  81
Reached Score:  81
Schedule:  [[0, 8], [7, 2], [3], [5, 8, 2], [4], [0], [4], [2], [7, 2], [4, 3, 8, 0], [6], [4, 8], [6], [], [5], [7, 3], [2], [6, 4], [3], [8], [2, 1], [7, 8], [6, 0, 7, 5, 8], [4, 7], [2], [6, 2, 0, 3, 8], [3, 4, 0], [], [2], [], [3], [8], [], [6, 0], [2], [], [2], [3, 6], [2, 0], [1], [7, 8], [5, 7, 2], [1], [8, 3, 0], [5]]


test 2:  7.60 sec


In [53]:
for i in range(len(sol2)):
    print(sol2[i])
    if((i+1)%3 ==0):
        print('\n')

[0, 8]
[7, 2]
[3]


[5, 8, 2]
[4]
[0]


[4]
[2]
[7, 2]


[4, 3, 8, 0]
[6]
[4, 8]


[6]
[]
[5]


[7, 3]
[2]
[6, 4]


[3]
[8]
[2, 1]


[7, 8]
[6, 0, 7, 5, 8]
[4, 7]


[2]
[6, 2, 0, 3, 8]
[3, 4, 0]


[]
[2]
[]


[3]
[8]
[]


[6, 0]
[2]
[]


[2]
[3, 6]
[2, 0]


[1]
[7, 8]
[5, 7, 2]


[1]
[8, 3, 0]
[5]




In [70]:
f2 = open('output2.txt','w')
for i in range(len(sol2)):
    if(len(sol2[i]) == 0):
        f2.write("empty ")
        if((i+1)%3 ==0):
            f2.write("\n")
        continue
    for j in range(len(sol2[i])):
        if(j != len(sol2[i])-1):
            f2.write(str(sol2[i][j])+',')
        else:
            f2.write(str(sol2[i][j]))
    f2.write(" ")
    if((i+1)%3 ==0):
        f2.write("\n")

f2.close()